In [1]:
import os.path as path
import numpy as np
import torch.nn.functional as functional
from IPython.display import display
from ipywidgets import FloatSlider, interactive
import ipywidgets as widgets
import matplotlib.pyplot as plt

from constants import *
from data_loaders import *
from model import CapsNet

%matplotlib inline

# Specifies model to load
MODEL = "model2.pt"

In [14]:
def reconstruct(prediction,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,c15):
    capsules[:,prediction,0,:] = c0
    reconstruction, _ = capsnet.decoder(capsules, data, target[i:i+1].cuda())
    
    print(capsules)
    print(target[i:i+1])
    print(target[i:i+1].max(dim=1)[1].reshape(-1,1))
    im = np.squeeze(reconstruction.data.cpu().numpy())
    im += abs(im.min())
    im /= im.max()
    plt.subplot(1,2,1)
    plt.imshow(im, cmap="gray");
    im2 = data[i, 0].data.cpu().numpy()
    im2 += abs(im.min())
    im2 /= im.max()
    plt.subplot(1,2,2)
    plt.imshow(im2, cmap="gray");

In [4]:
# Load model
capsnet = CapsNet(reconstruction_type="FC") # FC or Conv
if USE_GPU:
  capsnet.cuda()
model_path = path.join(SAVE_DIR, MODEL)
capsnet.load_state_dict(torch.load(model_path))

_, test_loader = load_mnist(BATCH_SIZE)
capsnet.eval()
data, target = iter(test_loader).next()
target = torch.eye(10).index_select(dim=0, index=target) # One-hot encode target
output, reconstruction, masked = capsnet(data.cuda())

In [15]:
i = 103 # index of chosen image in last batch
capsules = output[i:i+1] # capsules that correspond to this specific image

# Find prediction
classes = torch.sqrt((capsules**2).sum(2))
classes = functional.softmax(classes, dim=1)
_, prediction = classes.max(dim=1)

print("Target:{}".format(target[i:i+1,:].max(dim=1)[1].item()))
print("Prediction:{}".format(prediction.item()))

print(capsules[:,prediction,:,:].shape)
print(capsules[:,prediction,0,:].item())

MIN = -1
MAX = 1
STEP = 1e-2
CONTINUOUS_UPDATE = True

w = interactive(reconstruct,
                prediction=prediction,
                c0=FloatSlider(description="Capsule 0",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE),
                c1=FloatSlider(description="Capsule 1",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE),
                c2=FloatSlider(description="Capsule 2",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE),
                c3=FloatSlider(description="Capsule 3",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE),
                c4=FloatSlider(description="Capsule 4",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE),
                c5=FloatSlider(description="Capsule 5",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE),
                c6=FloatSlider(description="Capsule 6",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE),
                c7=FloatSlider(description="Capsule 7",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE),
                c8=FloatSlider(description="Capsule 8",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE),
                c9=FloatSlider(description="Capsule 9",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE),
                c10=FloatSlider(description="Capsule 10",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE),
                c11=FloatSlider(description="Capsule 11",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE),
                c12=FloatSlider(description="Capsule 12",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE),
                c13=FloatSlider(description="Capsule 13",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE),
                c14=FloatSlider(description="Capsule 14",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE),
                c15=FloatSlider(description="Capsule 15",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE))
display(w)

Target:4
Prediction:4
torch.Size([1, 1, 1, 16, 1])
0.0


interactive(children=(Dropdown(description='prediction', options=(tensor([ 4], device='cuda:0'),), value=tensor([ 4], device='cuda:0')), FloatSlider(value=0.0, description='Capsule 0', max=1.0, min=-1.0, step=0.01), FloatSlider(value=0.0, description='Capsule 1', max=1.0, min=-1.0, step=0.01), FloatSlider(value=0.0, description='Capsule 2', max=1.0, min=-1.0, step=0.01), FloatSlider(value=0.0, description='Capsule 3', max=1.0, min=-1.0, step=0.01), FloatSlider(value=0.0, description='Capsule 4', max=1.0, min=-1.0, step=0.01), FloatSlider(value=0.0, description='Capsule 5', max=1.0, min=-1.0, step=0.01), FloatSlider(value=0.0, description='Capsule 6', max=1.0, min=-1.0, step=0.01), FloatSlider(value=0.0, description='Capsule 7', max=1.0, min=-1.0, step=0.01), FloatSlider(value=0.0, description='Capsule 8', max=1.0, min=-1.0, step=0.01), FloatSlider(value=0.0, description='Capsule 9', max=1.0, min=-1.0, step=0.01), FloatSlider(value=0.0, description='Capsule 10', max=1.0, min=-1.0, step=0.01), FloatSlider(value=0.0, description='Capsule 11', max=1.0, min=-1.0, step=0.01), FloatSlider(value=0.0, description='Capsule 12', max=1.0, min=-1.0, step=0.01), FloatSlider(value=0.0, description='Capsule 13', max=1.0, min=-1.0, step=0.01), FloatSlider(value=0.0, description='Capsule 14', max=1.0, min=-1.0, step=0.01), FloatSlider(value=0.0, description='Capsule 15', max=1.0, min=-1.0, step=0.01), Output()), _dom_classes=('widget-interact',))